In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os, pickle, shutil, random, PIL

import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torchvision import models
from torch.utils.data import DataLoader,random_split,Dataset
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.transforms import v2

import matplotlib.pyplot as plt
import seaborn as sns

from training_utils import *
from focal_loss_with_smoothing import *

In [ ]:
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

DFNAME = 'resnet50'
device = torch.device('cuda:0')
criterion = FocalLossWithSmoothing(num_classes = 2, gamma=2, lb_smooth = 0.1)
# criterion2 =  nn.L1Loss()
# criterion1 = nn.CrossEntropyLoss()

modelname = 'resnet50'
n_epochs = 100
batch_size = 12

In [ ]:
# train_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/train_original/imgs/'
# test_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/test/imgs/'
# val_dir = '/home/aminul/CVL/cs791_project/malignant_vs_benign/val/imgs/'

train_dir = '/home/ahuq/Research/data/malignant_benign_1024/train/'
test_dir =  '/home/ahuq/Research/data/malignant_benign_1024/test/'

In [1]:
size = (1024,768)
train_set_whole = ImageFolder(train_dir,transform = transforms.Compose([
    # v2.RandomVerticalFlip(0.5),
    # v2.RandomHorizontalFlip(0.5),
    # v2.RandomRotation(15),
    # v2.RandomAutocontrast(0.5),
    # v2.RandomAdjustSharpness(2,0.5),
    v2.Resize(size),
    v2.ToTensor(),
    # v2.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
]))

# val_set = ImageFolder(val_dir,transform = transforms.Compose([
#     v2.Resize(size),
#     v2.ToTensor(),
#     # v2.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
# ]))

test_set = ImageFolder(test_dir,transform = transforms.Compose([
    v2.Resize(size),
    v2.ToTensor(),
    # v2.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
]))


train_set, valid_set = random_split(train_set_whole,[int(len(train_set_whole)*0.9), int(len(train_set_whole)*0.1)],
                                  generator=torch.Generator().manual_seed(0))

NameError: name 'ImageFolder' is not defined

In [ ]:
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers = 4)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers = 4)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers = 4)
test_loader_2 = DataLoader(test_set, batch_size=1, shuffle=False)

In [ ]:
# model = ResNet18().to(device)
model = torchvision.models.resnet50().to(device)
model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

In [ ]:
optim = torch.optim.Adam(model.parameters(),lr=0.0002, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optim, mode='min', factor=0.5, patience=3)

In [ ]:
history = {'train_loss': [], 'valid_loss': [],'train_acc':[],'valid_acc':[]}

for epoch in range(n_epochs):

    train_loss, train_acc = train(model,train_loader,criterion,optim,device,epoch)
    valid_loss, valid_acc = test(model,valid_loader, criterion, optim,modelname,device,epoch)

    scheduler.step(valid_loss)

    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    history['train_acc'].append(train_acc)
    history['valid_acc'].append(valid_acc)


with open('./storage/' + DFNAME + '.pkl', 'wb') as f:
    pickle.dump(history, f)

100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [00:25<00:00,  2.46it/s]


Epoch: [1]  loss: [3.03] Train Accuracy [62.87] 


100%|█████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.47it/s]


Epoch: [1]  loss: [2.52] Test Accuracy [69.51] 
Saving Best model...


100%|███████████████████████████████████████████████████████████████████████████████████| 62/62 [00:24<00:00,  2.51it/s]


...

Epoch: [42]  loss: [1.78] Train Accuracy [100.00] 


100%|█████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  3.48it/s]


Epoch: [42]  loss: [1.83] Test Accuracy [97.56] 


 10%|████████▏                                                                           | 6/62 [00:03<00:35,  1.59it/s]


KeyboardInterrupt: 

In [ ]:
new_model = torchvision.models.resnet50().to(device)
new_model.fc = torch.nn.Sequential(
    torch.nn.Linear(in_features=2048,
                    out_features=2, # same number of output units as our number of classes
                    bias=True)).to(device)

checkpoint = torch.load('./checkpoint/'+modelname+'model.pth.tar',map_location=torch.device('cpu'))
new_model.load_state_dict(checkpoint['model'])

_, _,y,y_pred =  best_test(new_model,test_loader,criterion,optim,device,1)


100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [00:03<00:00,  4.74it/s]

Epoch: [2]  loss: [2.85] Test Accuracy [64.18] 
